In [7]:
import polars as pl # similar pandas
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pyarrow.parquet as pq
import pyarrow as pa
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap